In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM

# Use an available LLaMA-like model (adjust path if needed)
model_path = "openlm-research/open_llama_3b"  # Substitute with the path of the model you're using
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Function to generate questions
def generate_question(topic, difficulty):
    """Generate a question dynamically using LLaMA based on topic and difficulty."""
    prompt = f"Generate a {difficulty} level question for {topic} and provide the correct answer."
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=80, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Splitting to get question and answer (assuming format: 'Question: ... Answer: ...')
    if "Answer:" in generated_text:
        question, correct_answer = generated_text.split("Answer:", 1)
    else:
        question, correct_answer = generated_text, ""
    return question.strip(), correct_answer.strip()

# Function to explain the correct answer
def explain_answer(question, correct_answer):
    """Provide an explanation for the correct answer."""
    prompt = f"Explain why the answer to '{question}' is '{correct_answer}' in simple terms."
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=100)
    explanation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return explanation

# Function to give adaptive feedback based on the user’s performance and confidence level
def give_dynamic_feedback(answer_correct, confidence):
    """Generate feedback based on the user's response accuracy and confidence."""
    if answer_correct and confidence == "low":
        prompt = "Encourage the user to be more confident in their answers."
    elif not answer_correct and confidence == "high":
        prompt = "Suggest double-checking answers in a positive way."
    else:
        prompt = "Give general feedback to motivate the user."
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=50)
    feedback = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return feedback

# Function to recommend resources dynamically
def recommend_dynamic_resources(topic, difficulty):
    """Provide learning resources based on topic and difficulty."""
    prompt = f"Recommend some {difficulty} level resources for {topic}."
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=100)
    resources = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resources

# Main quiz function
def quiz():
    print("Welcome to the Adaptive Quiz!")
    score = 0
    attempts = 0
    current_difficulty = "easy"
    subject_scores = {topic: {"score": 0, "attempts": 0} for topic in ["math", "science", "geography"]}

    while attempts < 5:  # Limit the number of questions
        topic = random.choice(list(subject_scores.keys()))  # Randomly choose a topic
        question, correct_answer = generate_question(topic, current_difficulty)

        # Present the question to the user
        print(f"Question: {question}")
        answer = input("Your Answer: ")

        # Check if user's answer is correct
        answer_correct = answer.lower().strip() == correct_answer.lower().strip()

        if answer_correct:
            print("Correct!")
            score += 1
            subject_scores[topic]["score"] += 1
        else:
            print(f"Incorrect! The correct answer is: {correct_answer}")
            explanation = explain_answer(question, correct_answer)
            print("Explanation:", explanation)

        # Get user's confidence and give feedback
        confidence = input("How confident are you in your answer? (High/Medium/Low): ").lower()
        feedback = give_dynamic_feedback(answer_correct, confidence)
        print(feedback)

        # Update difficulty based on performance
        if score % 2 == 0 and current_difficulty != "hard":
            current_difficulty = "medium" if current_difficulty == "easy" else "hard"
        elif not answer_correct and attempts % 2 == 1 and current_difficulty != "easy":
            current_difficulty = "medium" if current_difficulty == "hard" else "easy"

        # Track subject attempts
        subject_scores[topic]["attempts"] += 1
        attempts += 1

        print(f"Your current score is: {score}/{attempts}")
        print("Resources to help you improve:")
        print(recommend_dynamic_resources(topic, current_difficulty))

    # Final summary and suggestions
    print("\nQuiz completed!")
    print(f"Your final score is: {score}/{attempts}")

    # Display subject-wise performance and additional resources
    for topic, data in subject_scores.items():
        subject_score = data["score"]
        subject_attempts = data["attempts"]
        accuracy = subject_score / subject_attempts if subject_attempts > 0 else 0
        print(f"{topic.capitalize()} Performance: {subject_score}/{subject_attempts} correct (Accuracy: {accuracy:.0%})")
        level = "easy" if accuracy < 0.5 else "medium" if accuracy < 0.8 else "hard"
        print("Additional resources to help you improve in", topic)
        print(recommend_dynamic_resources(topic, level))

if _name_ == "_main_":
    quiz()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

In [ ]:
import random

# Sample questions categorized by difficulty level
questions = {
    "easy": [
        {"question": "What is 2 + 2?", "answer": "4", "topic": "math"},
        {"question": "What color is the sky?", "answer": "blue", "topic": "science"}
    ],
    "medium": [
        {"question": "What is the capital of France?", "answer": "Paris", "topic": "geography"},
        {"question": "What is 10 * 5?", "answer": "50", "topic": "math"}
    ],
    "hard": [
        {"question": "What is the derivative of x^2?", "answer": "2x", "topic": "math"},
        {"question": "What is the square root of 144?", "answer": "12", "topic": "math"}
    ]
}

# Resources categorized by topic and difficulty
resources = {
    "math": {
        "easy": ["Khan Academy Basics in Math", "BBC Bitesize Math for beginners"],
        "medium": ["Coursera Intermediate Math", "Khan Academy Math Intermediate"],
        "hard": ["MIT OpenCourseWare in Calculus", "Khan Academy Advanced Math Lessons"]
    },
    "science": {
        "easy": ["Khan Academy Basics in Science", "BBC Bitesize for beginners"],
        "medium": ["Coursera Science Intermediate Courses", "Khan Academy Science Intermediate"],
        "hard": ["MIT OpenCourseWare in Science", "Khan Academy Advanced Science Lessons"]
    },
    "geography": {
        "easy": ["Geography Basics on BBC Bitesize", "National Geographic Beginner Lessons"],
        "medium": ["Coursera Geography Courses", "Geography Intermediate on Khan Academy"],
        "hard": ["Advanced Geography Resources on MIT OpenCourseWare", "Advanced World Geography"]
    }
}

def get_question(difficulty):
    """Get a random question from the specified difficulty level."""
    return random.choice(questions[difficulty])

def give_feedback(score, attempts, current_difficulty):
    """Provide feedback based on performance so far."""
    accuracy = score / attempts if attempts > 0 else 0
    if accuracy >= 0.8:
        return f"Excellent! You’re mastering the {current_difficulty} level."
    elif 0.5 <= accuracy < 0.8:
        return f"Good effort! Keep practicing to improve your {current_difficulty} skills."
    else:
        return f"Don't worry! Reviewing some basics could help you improve at the {current_difficulty} level."

def recommend_resources(topic, difficulty):
    """Suggest resources based on the current topic and difficulty."""
    print(f"Recommended resources for {topic} at {difficulty} level:")
    for resource in resources[topic][difficulty]:
        print(f" - {resource}")

def quiz():
    print("Welcome to the Adaptive Quiz!")
    score = 0
    attempts = 0
    current_difficulty = "easy"
    incorrect_topics = {}

    while attempts < 5:  # Limit the number of questions
        q = get_question(current_difficulty)
        answer = input(q["question"] + " ")
        confidence = input("How confident are you in your answer? (High/Medium/Low): ").lower()

        if answer.lower() == q["answer"].lower():
            print("Correct!")
            score += 1
            # Increase difficulty after 2 correct answers
            if score % 2 == 0 and current_difficulty != "hard":
                current_difficulty = "medium" if current_difficulty == "easy" else "hard"
        else:
            print(f"Incorrect! The correct answer is: {q['answer']}")
            # Track topics with incorrect answers
            if q["topic"] in incorrect_topics:
                incorrect_topics[q["topic"]] += 1
            else:
                incorrect_topics[q["topic"]] = 1
            # Decrease difficulty after 2 incorrect answers
            if score > 0 and attempts % 2 == 1:
                current_difficulty = "easy" if current_difficulty == "medium" else "medium"

        # Provide feedback based on confidence level and accuracy
        if confidence == "low" and answer.lower() == q["answer"].lower():
            print("You got it right! Be confident in your abilities!")
        elif confidence == "high" and answer.lower() != q["answer"].lower():
            print("That's okay! Double-check your responses next time.")

        attempts += 1
        print(f"Your current score is: {score}/{attempts}")
        print(give_feedback(score, attempts, current_difficulty))
        recommend_resources(q["topic"], current_difficulty)  # Suggest resources based on topic and difficulty

    # Final summary and suggestions
    print("\nQuiz completed!")
    print(f"Your final score is: {score}/{attempts}")
    performance = score / attempts if attempts > 0 else 0

    if performance >= 0.8:
        print("Outstanding performance! Keep up the great work by exploring more challenging materials.")
    elif 0.5 <= performance < 0.8:
        print("Good job! To improve, review intermediate-level materials and challenge yourself with harder questions.")
    else:
        print("Keep practicing! Starting with the basics will strengthen your foundation.")

    # Topic-specific feedback based on incorrect answers
    if incorrect_topics:
        print("\nAreas to Improve:")
        for topic, count in incorrect_topics.items():
            print(f"- {topic.capitalize()}: You missed {count} question(s). Here are some resources to review:")
            recommend_resources(topic, "easy" if performance < 0.5 else "medium")

if __name__ == "__main__":
    quiz()

Welcome to the Adaptive Quiz!
What is 2 + 2? 4
How confident are you in your answer? (High/Medium/Low): High
Correct!
Your current score is: 1/1
Excellent! You’re mastering the easy level.
Recommended resources for math at easy level:
 - Khan Academy Basics in Math
 - BBC Bitesize Math for beginners
What color is the sky? Blue
How confident are you in your answer? (High/Medium/Low): Medium
Correct!
Your current score is: 2/2
Excellent! You’re mastering the medium level.
Recommended resources for science at medium level:
 - Coursera Science Intermediate Courses
 - Khan Academy Science Intermediate
What is the capital of France? Paris
How confident are you in your answer? (High/Medium/Low): Low
Correct!
You got it right! Be confident in your abilities!
Your current score is: 3/3
Excellent! You’re mastering the medium level.
Recommended resources for geography at medium level:
 - Coursera Geography Courses
 - Geography Intermediate on Khan Academy
What is the capital of France? Paris
How 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()